In [54]:
import math

In [59]:
class Value:
    def __init__(self, data, _operation = "", _children = (), label = ""):
        self.data = data
        self._prev = set(_children)
        self._operation = _operation
        self.label = label
        
    def __repr__(self):
        return f"Value(data = {self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        operation = " add "
        children = (self, other)
        addition = self.data + other.data
        result = Value(data = addition, _operation = operation, _children = children)
        return result

    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        operation = " mul "
        children = (self, other)
        addition = self.data * other.data
        result = Value(data = addition, _operation = operation, _children = children)
        return result

    def __rmul__(self, other):
        return self * other

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __neg__(self):
        return -1 * self

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "power can be integer or float"
        operation = f" pow({other})"
        children = (self, )
        power = self.data ** other
        result = Value(data = power, _children = children, _operation = operation)
        return result

    def __truediv__(self, other):
        return self * (other ** -1)

    def __rtruediv__(self, other):
        return other * (self ** -1)

    def exp(self):
        operation = " exp "
        children = (self, )
        e = math.exp(self.data)
        result = Value(data = e, _operation = operation, _children = children)
        return result

    def tanh(self):
        x = self.data
        operation = " tanh "
        children = (self, )
        tan_h = (math.exp(2.0 * x) - 1.0) / (math.exp(2.0 * x) + 1.0)
        result = Value(data = tan_h, _operation = operation, _children = children)
        return result

In [62]:
a = Value(10.0, label = "a")
b = (a).tanh()

In [63]:
b

Value(data = 0.9999999958776927)